### III ) Création d'un dataframe de reviews par compagnie aérienne
df_compagnies.csv -> Reviews_Emirates.csv, Reviews_AEGEAN.csv, ...

In [1]:
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import time
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
df_compagnies = pd.read_csv('df_compagnies.csv', sep='§', engine='python', index_col=0, encoding='utf-8')
df_compagnies

,Compagnie,link
0,Emirates,https://www.tripadvisor.fr/Airline_Review-d872...
1,AEGEAN,https://www.tripadvisor.fr/Airline_Review-d872...
2,Qatar Airways,https://www.tripadvisor.fr/Airline_Review-d872...
3,Turkish Airlines,https://www.tripadvisor.fr/Airline_Review-d872...
4,Singapore Airlines,https://www.tripadvisor.fr/Airline_Review-d872...
...,...,...
235,Nordwind Airlines,https://www.tripadvisor.fr/Airline_Review-d108...
236,Buddha Air,https://www.tripadvisor.fr/Airline_Review-d872...
237,Lucky Air,https://www.tripadvisor.fr/Airline_Review-d872...
238,Ernest,https://www.tripadvisor.fr/Airline_Review-d150...


In [3]:
df_comp_reviews = pd.DataFrame(columns=["user_name","location","tags","date_voyage","commentaire","date_commentaire",
                                        "contributions","votes utiles","Espace pour les jambes","Confort du siège",
                                        "Divertissement à bord","Service client","Rapport qualité/prix","Propreté",
                                        "Enregistrement et embarquement","Restauration et boissons","Vol"])
df_comp_reviews

,user_name,location,tags,date_voyage,commentaire,date_commentaire,contributions,votes utiles,Espace pour les jambes,Confort du siège,Divertissement à bord,Service client,Rapport qualité/prix,Propreté,Enregistrement et embarquement,Restauration et boissons,Vol


In [4]:
options = Options()
#options.headless = True

driver = webdriver.Firefox(options=options)

In [5]:
comp_name, comp_url = df_compagnies.loc[1,:]
comp_name, comp_url

('AEGEAN',
 'https://www.tripadvisor.fr/Airline_Review-d8728985-Reviews-AEGEAN#REVIEWS')

In [6]:
language = 'en'

In [7]:
driver.get(comp_url)

In [8]:
for i in range(2):
    
    wait_time_long = 5 + np.random.random(1)[0]*5
    wait_time_small = 0.5 + np.random.random(1)[0]
    
    # wait for loading to finish
    print("wait", wait_time_long, ", loading page...")
    time.sleep(wait_time_long)
    
    # select the rigth language if not selected
    btn_langages = driver.find_elements_by_xpath("//div[@class='ui_column  is-3-tablet is-shown-at-tablet ']\
                    //ul[contains(@class,'location-review-review-list-parts-ReviewFilter__filter_table')]\
                    //li")

    for li in btn_langages:
        checked = li.find_elements_by_xpath(".//input")
        if checked : checked = checked[0].get_attribute("checked")
        if checked : checked = True
        else : checked = False

        langue = li.find_elements_by_xpath(".//input")
        if langue : langue = langue[0].get_attribute("value")

        if (langue == language) and (checked == False):
            btn = li.find_elements_by_xpath(".//span")
            if btn : 
                btn = btn[0].click()
                time.sleep(wait_time_long)
                
            
    # go to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    time.sleep(wait_time_small)
    
    # load comments details
    boutons = driver.find_elements_by_xpath("//*[contains(@class,'ui_icon caret-down location-review-review-list-parts-ExpandableReview')]")
    if boutons : 
        if boutons[0].is_displayed() :
            boutons[0].click()
            time.sleep(wait_time_small)
            
    results = driver.find_elements_by_xpath("//div[@data-tab='TABS_REVIEWS'][1]\
                                            //div[@class=''][1]\
                                            //*[contains(@class, 'location-review-card-Card__ui_card')]")
    print('Page', i,', number of comments :', len(results))
    
    # for every block of comment
    for block in results:
        
        # block 1
        block_user_info = block.find_elements_by_xpath("div[contains(@class,'social-member-event-MemberEventOnObjectBlock')]")[0]

        user_name = block_user_info.find_elements_by_xpath("div[1]/div[2]/span/a")
        if user_name : user_name = user_name[0].get_attribute('href').split('/')[-1]
        else : user_name = np.nan

        user_location = block_user_info.find_elements_by_xpath("div[1]/div[3]/span/span[contains(@class,'MemberHometown__hometown')]")
        if user_location: user_location = user_location[0].text
        else : user_location = np.nan

        date_commentaire = block_user_info.find_elements_by_xpath("div[1]/div[2]/span")
        if date_commentaire : 
            date_commentaire = date_commentaire[0].get_attribute('innerHTML').split('</a>')[-1].split(' a écrit un avis le ')[-1]
            if date_commentaire.lower() == "hier" : date_commentaire = (datetime.today() - timedelta(days=1)).strftime('%d %B')
            if date_commentaire.lower() == "aujourd'hui" : date_commentaire = datetime.today().strftime('%d %B')
        else : date_commentaire = np.nan

        user_stats = block_user_info.find_elements_by_xpath("div[1]/div[3]/span[contains(@class,'MemberHeaderStats__stat_item')]/span")
        dict_stats = {}
        for us in user_stats:
            stat_name = us.get_attribute('innerHTML').split('</span>')[-1].strip()
            stat_value = us.find_elements_by_xpath("span")[0].text
            dict_stats[stat_name] = stat_value

        # block 2
        block_user_review = block.find_elements_by_xpath("div[contains(@class,'location-review-review-list-parts-SingleReview__mainCol')]")[0]

        tags = block_user_review.find_elements_by_xpath("div[contains(@class,'location-review-review-list-parts-RatingLine')]")
        if tags : tags = tags[0].text.split("\n")
        else : tags = np.nan

        titre = block_user_review.find_elements_by_xpath("div[@data-test-target='review-title']")
        if titre: titre = titre[0].text
        else : titre = np.nan

        date_voyage = block_user_review.find_elements_by_xpath(".//span[contains(@class,'location-review-review-list-parts-EventDate__event_date')]")
        if date_voyage : date_voyage = date_voyage[0].get_attribute('innerHTML').split("</span>")[-1].strip()
        else : date_voyage = np.nan

        commentaire = block_user_review.find_elements_by_xpath(".//q[contains(@class,'location-review-review-list-parts-ExpandableReview__reviewText')]")
        if commentaire : commentaire = commentaire[0].text
        else : commentaire = np.nan

        vol_note = block_user_review.find_elements_by_xpath(".//div[contains(@class,'location-review-review-list-parts-RatingLine')]/span")
        if vol_note : vol_note = vol_note[0].get_attribute('class').split('_')[-1]
        else : vol_note = np.nan

        notes = block_user_review.find_elements_by_xpath(".//div[contains(@class,'location-review-review-list-parts-AdditionalRatings__ratings')]/div")
        dict_notes = {}
        for e in notes:
            categorie_name = e.find_elements_by_xpath("span")[1].text
            categorie_note = e.find_elements_by_xpath("span/span")[0].get_attribute('class').split('_')[-1]
            dict_notes[categorie_name] = categorie_note
        dict_notes['Vol'] = vol_note
        
        # ajout des infos dans le dataframe
        all_values = {"user_name" : user_name,
                      "location" : user_location,
                      "tags" : tags,
                      "date_voyage" : date_voyage,
                      "commentaire" : commentaire,
                      "date_commentaire" : date_commentaire}
        all_values.update(dict_stats)
        all_values.update(dict_notes)

        df_comp_reviews = df_comp_reviews.append(pd.Series(all_values), ignore_index=True, sort=False)

    df_comp_reviews.to_csv('Reviews_'+comp_name+'.csv', sep='§', encoding='utf-8')
    
    suivant = driver.find_elements_by_xpath("//*//a[@class='ui_button nav next primary ']")
    if suivant : 
        suivant = suivant[0].click()

wait 6.990390829485472 , loading page...
Page 0 , number of comments : 5
wait 6.449950194544945 , loading page...
Page 1 , number of comments : 5


In [9]:
df_comp_reviews

,user_name,location,tags,date_voyage,commentaire,date_commentaire,contributions,votes utiles,Espace pour les jambes,Confort du siège,Divertissement à bord,Service client,Rapport qualité/prix,Propreté,Enregistrement et embarquement,Restauration et boissons,Vol,vote utile
0,suehY7409QF,"Londres, Royaume-Uni","[Londres-Héraklion, Europe, Économique]",décembre 2019,Very basic service especially for this length ...,16 December,4,NaN,30,30,10,30,20,30,30,10,30,NaN
1,florentinaa312,NaN,"[Londres-Kárpathos, Europe, Économique]",décembre 2019,in 2018 we flew with Aegean and we had a very ...,16 December,5,NaN,10,10,10,10,10,10,10,NaN,10,NaN
2,KennethSeet1975,"Singapore, Singapore","[Thessalonique-Francfort, Europe, Économique]",décembre 2019,Travelled on Aegean Airlines from Thessaloniki...,15 déc.,311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,1
3,akisroussis,"Athènes, Grèce","[Athènes-Canton de Genève, Europe, Économique]",décembre 2019,For another european trip we decided to fly wi...,13 déc.,22,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,NaN
4,457gemmah,NaN,"[Londres-Larnaca, Europe, Affaires]",janvier 2019,I cant fault this airline. Admittedly things c...,12 déc.,5,4,50,50,50,50,50,40,50,50,50,NaN
5,Andreasdan,"Larnaca, Chypre","[Paris-Athènes, Europe, Économique]",décembre 2019,It was a great flight. Very smoth get in the a...,12 déc.,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,NaN
6,KostasG308,NaN,"[Athènes-Zurich, Europe, Affaires]",décembre 2019,Traveled recently and first time business clas...,11 déc.,13,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,NaN
7,959lefterisl,"Thessalonique, Grèce","[Düsseldorf-Thessalonique, Europe, Économique]",novembre 2019,"excellent experience, because we travel with o...",11 déc.,2,NaN,50,50,40,50,50,50,50,50,50,NaN
8,redman5314,"Egaleo, Grèce","[Athènes-Berlin, Europe, Économique]",octobre 2019,I am a frequent flyer and I am very glad of th...,11 déc.,3 083,18 477,30,50,30,50,50,50,50,40,50,NaN
9,margaritamanousou1,NaN,"[Athènes-Héraklion, Intérieur, Économique]",décembre 2019,GOOD FLIGHT - GOOD SERVICE - NO PROBLEMS\nThis...,11 déc.,18,3,30,30,30,30,20,30,30,30,40,NaN
